In [1]:
import numpy as np
from qecdec import BPDecoder, DMemBPDecoder, RotatedSurfaceCode
from beliefmatching import detector_error_model_to_check_matrices
from time import time

In [2]:
d = 7
p = 0.005
num_shots = 10_000

code = RotatedSurfaceCode(d=d)
circuit = code.make_circuit_memory_z_experiment(
    rounds=d,
    data_qubit_error_rate=p,
    meas_error_rate=p,
    prep_error_rate=p,
    gate1_error_rate=p,
    gate2_error_rate=p,
    keep_z_detectors_only=True
)
detectors_sample, observables_sample = circuit.compile_detector_sampler(seed=0).sample(num_shots, separate_observables=True)
detectors_sample = detectors_sample.astype(np.uint8)
observables_sample = observables_sample.astype(np.uint8)

dem = circuit.detector_error_model()
matrices = detector_error_model_to_check_matrices(dem)
check_matrix, observables_matrix, priors = matrices.check_matrix, matrices.observables_matrix, matrices.priors
check_matrix = check_matrix.toarray().astype(np.uint8)
observables_matrix = observables_matrix.toarray().astype(np.uint8)
priors = priors.astype(np.float64)

In [3]:
bp = BPDecoder(
    pcm=check_matrix,
    prior=priors,
    max_iter=50
)

t1 = time()
ehat = bp.decode_batch(detectors_sample)
t2 = time()
print(f"BP decoder time: {t2 - t1} seconds")

detectors_predict = (ehat @ check_matrix.T) % 2
num_nonconverged = np.sum(np.any(detectors_predict != detectors_sample, axis=1))
print(f"Number of nonconverged: {num_nonconverged}")

observables_predict = (ehat @ observables_matrix.T) % 2
num_logical_errors = np.sum(np.any(observables_predict != observables_sample, axis=1))
print(f"Number of logical errors: {num_logical_errors}")

BP decoder time: 4.979511022567749 seconds
Number of nonconverged: 2069
Number of logical errors: 695


In [4]:
for seed in range(10):
    print(f"Seed: {seed}")
    np.random.seed(seed)
    gamma = np.random.uniform(-0.2, 0.9, check_matrix.shape[1])
    # gamma = np.random.uniform(-0.1, 0.9, check_matrix.shape[1])

    dmembp = DMemBPDecoder(
        pcm=check_matrix,
        prior=priors,
        gamma=gamma,
        max_iter=50
    )

    ehat = dmembp.decode_batch(detectors_sample)

    detectors_predict = (ehat @ check_matrix.T) % 2
    num_nonconverged = np.sum(np.any(detectors_predict != detectors_sample, axis=1))
    print(f"Number of nonconverged: {num_nonconverged}")

    observables_predict = (ehat @ observables_matrix.T) % 2
    num_logical_errors = np.sum(np.any(observables_predict != observables_sample, axis=1))
    print(f"Number of logical errors: {num_logical_errors}")

Seed: 0
Number of nonconverged: 1014
Number of logical errors: 623
Seed: 1
Number of nonconverged: 919
Number of logical errors: 599
Seed: 2
Number of nonconverged: 1000
Number of logical errors: 623
Seed: 3
Number of nonconverged: 951
Number of logical errors: 566
Seed: 4
Number of nonconverged: 886
Number of logical errors: 549
Seed: 5
Number of nonconverged: 832
Number of logical errors: 531
Seed: 6
Number of nonconverged: 870
Number of logical errors: 599
Seed: 7
Number of nonconverged: 910
Number of logical errors: 606
Seed: 8
Number of nonconverged: 963
Number of logical errors: 625
Seed: 9
Number of nonconverged: 1004
Number of logical errors: 599
